As a NCAA outsider, I am super interested in a question that are there safe bets between two teams looking at their historical confrontations. For example, team A has been beating team B since day one.

Beyond that, the percentage of team A beating team B historically would also be a fairly good feature for modelling. 

It should be useful as a prior for Bayes theorem models.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

As a win-lose only study, this study needs only the compact results.

In [ ]:
rdat = pd.read_csv('../input/RegularSeasonCompactResults.csv')
tdat = pd.read_csv('../input/TourneyCompactResults.csv')
dat = pd.concat([rdat, tdat])

In [ ]:
# only need to know who won and who lost
games = dat.loc[:, ['Wteam', 'Lteam']].copy()

In [ ]:
# create head-to-head combination regardless of game result
games.loc[:, 'Min_team'] = games.loc[:, ['Wteam', 'Lteam']].min(axis=1)
games.loc[:, 'Max_team'] = games.loc[:, ['Wteam', 'Lteam']].max(axis=1)
games.loc[:, 'game'] = games.Min_team.apply(str) + ', ' + games.Max_team.apply(str)

In [ ]:
games.head()

In [ ]:
# create head-to-head combination that contains info of game results
games.loc[:, 'win_lose'] = games.Wteam.apply(str) + ', ' + games.Lteam.apply(str)

In [ ]:
games.head()

It's a coincidence that in the first five records all winning teams have smaller id, which surely shouldn't be an assumption.

In [ ]:
gpd_win = games.groupby(['game', 'win_lose'])
gpd_game = games.groupby('game')

In [ ]:
game_counts = gpd_game.size()
win_counts = gpd_win.size()

In [ ]:
game_counts.tail()

In [ ]:
win_counts.tail()

In the example above, you can see team 1460 and 1464 have contested 46 time. Team 1460 won 37 of them.

A problem is if two teams have faced each other for limited times, the result may not indicate anything. So I want to set a lower limit to qualify the games I need to look at.

In [ ]:
min_ncontests = 20
win_rate = win_counts.div(game_counts.loc[game_counts >= min_ncontests], level=0).dropna() * 100

In [ ]:
win_rate.tail()

So team 1460's 37 wins turns into a 80% winning rate.

Now I want to see which team always beats another team.

In [ ]:
always_win = win_rate.loc[win_rate >= 100]
always_win.name = 'percentage'
always_win = always_win.reset_index()
always_win

It's interesting to see there are 7 pairs. Wondering who they are.

In [ ]:
always_win.loc[:, 'Wteam'] = always_win.win_lose.str.extract(r'(\d+), \d+', expand=False)
always_win.loc[:, 'Lteam'] = always_win.win_lose.str.extract(r'\d+, (\d+)', expand=False)

In [ ]:
always_win

In [ ]:
teams = pd.read_csv('../input/Teams.csv').set_index('Team_Id')
teams.head()

In [ ]:
always_win.loc[:, 'Wteam'] = always_win.Wteam.astype('int').map(teams.Team_Name)
always_win.loc[:, 'Lteam'] = always_win.Lteam.astype('int').map(teams.Team_Name)

In [ ]:
always_win

OK. Now you can go ahead and pull out all their technical stats for their matches to have a detailed study why e.g. Arizona beat and never lost to Northern Arizona 20+ times.

Suppose it will give you clues that what are important factors for a team to keep winning.